# Method Resolution Order

To obtain an `MRO`, the interpreter method resolution algorithm carries out a left to right depth first listing of all classes in the hierarchy. 

In the trivial example above, this results in the following class list, `[D, B, A, object, C, A, object]` . Note that all objects will inherit from the root object class if no parent class is supplied during class definition. Finally, for each class that occurs multiple times, all occurrences are removed except the last occurrence resulting in an MRO of `[D, B, C, A, object]` for the previous class hierarchy. This result is the order in which classes would be searched for attributes for a given instance of `D`.

In [1]:
>>> class A(object):
...     def m(self):
...         print("m of A called")
>>> class B(A):
...     def m(self):
...         print("m of B called")
>>> class C(A):
...     def m(self):
...         print("m of C called")
>>> class D(B,C):
...     def m(self):
...         print("m of D called")

Let's apply the method m on an instance of `D`. We can see that only the code of the method `m` of `D` will be executed. We can also explicitly call the methods m of the other classes via the class name.

In [2]:
x = D()
D.m(x)
B.m(x)
C.m(x)
A.m(x)

m of D called
m of B called
m of C called
m of A called


Now let's assume that the method m of `D` should execute the code of `m` of `B`, `C` and `A` as well, when it is called. We could implement it like this:

In [3]:
>>> class D(B,C):
...     def m(self):
...         print("m of D called")
...         B.m(self)
...         C.m(self)
...         A.m(self)

In [4]:
x = D()

In [5]:
x.m()

m of D called
m of B called
m of C called
m of A called


### **Let's complicate**

But it turns out once more that things are more complicated than it seems.

How can we cope with the situation, if both `m` of `B` and `m` of `C` will have to call `m` of `A` as well?

In this case, we have to take away the call `A.m(self)` from `m` in `D`.

The code might look like this, but there is still a bug lurking in it.

In [6]:
>>> class A:
...     def m(self):
...         print("m of A called")

>>> class B(A):
...     def m(self):
...         print("m of B called")
...         A.m(self)

>>> class C(A):
...     def m(self):
...         print("m of C called")
...         A.m(self)

>>> class D(B,C):
...     def m(self):
...         print("m of D called")
...         B.m(self)
...         C.m(self)

**Now the method `m` of `A` will be called twice**

In [9]:
x = D()

In [10]:
x.m()

m of D called
m of B called
m of A called
m of C called
m of A called


### **Non-Pythonic way of solution to fix this**


One way to solve this problem 

By splitting the methods `m` of `B` and `C` in two methods.

The first method, called `_m` consists of the specific code for `B` and `C` and the other method is still called `m`, but consists now of a call `self._m()` and a call `A.m(self)`.

The code of the method `m` of `D` consists now of the specific code of `D` `print("m of D called")`, and the calls `B._m(self)`, `C._m(self)` and `A.m(self)`.


In [11]:
>>> class A:
...     def m(self):
...         print("m of A called")

>>> class B(A):
...     def _m(self):
...         print("m of B called")
...     def m(self):
...         self._m()
...         A.m(self)

>>> class C(A):
...     def _m(self):
...         print("m of C called")
...     def m(self):
...         self._m()
...         A.m(self)

>>> class D(B,C):
...     def m(self):
...         print("m of D called")
...         B._m(self)
...         C._m(self)
...         A.m(self)

In [14]:
x = D(); x.m();

m of D called
m of B called
m of C called
m of A called


### **Our problem is solved :), but still it is confusing.**

The optimal way to solve the problem, which is the **`super`** pythonic way, consists in calling the super function

In [15]:
>>> class A(object):
...     def m(self):
...         print("m of A called")
>>> class B(A):
...     def m(self):
...         print("m of B called")
...         super().m()
>>> class C(A):
...     def m(self):
...         print("m of C called")
...         super().m()
>>> class D(B,C):
...     def m(self):
...         print("m of D called")
...         super().m()

In [16]:
x = D()

In [17]:
x.m()

m of D called
m of B called
m of C called
m of A called


The **`super`** function is often used when instances are initialized with the **`__init__`** method.

In [18]:
>>> class A(object):
...     def __init__(self):
...         print("A.__init__")

>>> class B(A):
...     def __init__(self):
...         print("B.__init__")
...         super().__init__()

>>> class C(A):
...     def __init__(self):
...         print("C.__init__")
...         super().__init__()

>>> class D(B,C):
...     def __init__(self):
...         print("D.__init__")
...         super().__init__()

In [19]:
x = D()

D.__init__
B.__init__
C.__init__
A.__init__


In [20]:
d = D()

D.__init__
B.__init__
C.__init__
A.__init__


In [21]:
c = C()

C.__init__
A.__init__


In [22]:
b = B()

B.__init__
A.__init__


In [23]:
a = A()

A.__init__


## **Important Note**

### The question arises:
- How the super functions makes its decision?
- How does it decide which class has to be used? 

As we have already mentioned, it uses the so-called **`Method Resolution Order(MRO)`**. It is based on the **C3 superclass linearization** algorithm. This is called a **linearization**, because the tree structure is broken down into a linear order. The **MRO** method can be used to create this list.

In [24]:
D.mro()

[__main__.D, __main__.B, __main__.C, __main__.A, object]

In [25]:
C.mro()

[__main__.C, __main__.A, object]

In [26]:
B.mro()

[__main__.B, __main__.A, object]

In [27]:
A.mro()

[__main__.A, object]